In [218]:
import pandas as pd
import numpy as np
import unicodedata

## Lectura de todos los CSV

In [ ]:
exoplanetas = pd.read_csv('exoplanet.eu.csv', low_memory=False)

In [165]:
# 1️⃣ STELLARHOSTS_2025.csv (estrellas anfitrionas)
estrellas_anfitrionas = pd.read_csv(
    'STELLARHOSTS_2025.csv',
    comment='#',          # Ignora líneas de comentarios
    low_memory=False,
    encoding='utf-8',
    on_bad_lines='skip'   # Ignora líneas mal formadas
)
print("STELLARHOSTS columnas:", estrellas_anfitrionas.columns.tolist())
print("Filas:", len(estrellas_anfitrionas))

# 2️⃣ PS_2025.csv (planetas / sistemas planetarios)
planetas = pd.read_csv(
    'PS_2025.csv',
    comment='#',
    low_memory=False,
    encoding='utf-8',
    on_bad_lines='skip'
)
print("PS columnas:", planetas.columns.tolist())
print("Filas:", len(planetas))

# 3️⃣ k2pandc_2025.csv (planetas y candidatos K2)
planetas_candidatos = pd.read_csv(
    'k2pandc_2025.csv',
    comment='#',
    low_memory=False,
    encoding='utf-8',
    on_bad_lines='skip'
)
print("K2PANDC columnas:", planetas_candidatos.columns.tolist())
print("Filas:", len(planetas_candidatos))

STELLARHOSTS columnas: ['sy_name', 'hostname', 'sy_snum', 'sy_pnum', 'st_refname', 'st_spectype', 'st_teff', 'st_tefferr1', 'st_tefferr2', 'st_tefflim', 'st_rad', 'st_raderr1', 'st_raderr2', 'st_radlim', 'st_mass', 'st_masserr1', 'st_masserr2', 'st_masslim', 'st_met', 'st_meterr1', 'st_meterr2', 'st_metlim', 'st_metratio', 'st_logg', 'st_loggerr1', 'st_loggerr2', 'st_logglim', 'sy_refname', 'rastr', 'ra', 'decstr', 'dec', 'sy_dist', 'sy_disterr1', 'sy_disterr2', 'sy_vmag', 'sy_vmagerr1', 'sy_vmagerr2', 'sy_kmag', 'sy_kmagerr1', 'sy_kmagerr2', 'sy_gaiamag', 'sy_gaiamagerr1', 'sy_gaiamagerr2']
Filas: 46887
PS columnas: ['pl_name', 'hostname', 'default_flag', 'sy_snum', 'sy_pnum', 'discoverymethod', 'disc_year', 'disc_facility', 'soltype', 'pl_controv_flag', 'pl_refname', 'pl_orbper', 'pl_orbpererr1', 'pl_orbpererr2', 'pl_orbperlim', 'pl_orbsmax', 'pl_orbsmaxerr1', 'pl_orbsmaxerr2', 'pl_orbsmaxlim', 'pl_rade', 'pl_radeerr1', 'pl_radeerr2', 'pl_radelim', 'pl_radj', 'pl_radjerr1', 'pl_radje

## Limpieza "Exoplanetas"

Columnas

In [166]:
columnas_importantes = [
    "name", "planet_status", "mass", "mass_sini", "radius",
    "orbital_period", "semi_major_axis", "eccentricity", "inclination",
    "temp_calculated", "temp_measured", "molecules",
    "star_name", "star_distance", "star_mass", "star_radius", "star_teff",
    "detection_type", "mass_measurement_type", "radius_measurement_type"
]

# 1) Eliminar todas las columnas que no nos importan
exoplanetas = exoplanetas[[col for col in exoplanetas.columns if col in columnas_importantes]]

# 2) Crear columnas unificadas para masa y temperatura
exoplanetas["mass_final"] = exoplanetas["mass"].fillna(exoplanetas["mass_sini"])
exoplanetas["temp_final"] = exoplanetas["temp_measured"].fillna(exoplanetas["temp_calculated"])

# 3) Eliminar filas donde falten los 2 parámetros más útiles (masa, radio, temperatura)
exoplanetas = exoplanetas.dropna(subset=["mass_final", "radius", "temp_final"])

# 4) Resetear índice tras la limpieza
exoplanetas = exoplanetas.reset_index(drop=True)

# 5) Ver cuántos datos quedan después del filtrado
print("✅ Dataset limpio. Filas restantes:", len(exoplanetas))
print("📌 Columnas que quedaron:", exoplanetas.columns.tolist())

✅ Dataset limpio. Filas restantes: 1180
📌 Columnas que quedaron: ['name', 'planet_status', 'mass', 'mass_sini', 'radius', 'orbital_period', 'semi_major_axis', 'eccentricity', 'inclination', 'temp_calculated', 'temp_measured', 'detection_type', 'mass_measurement_type', 'radius_measurement_type', 'molecules', 'star_name', 'star_distance', 'star_mass', 'star_radius', 'star_teff', 'mass_final', 'temp_final']


Detectar número de missing values

In [167]:
print(exoplanetas.isna().sum())

missing_percent = exoplanetas.isna().sum() / len(exoplanetas) * 100
print(missing_percent.sort_values(ascending=False))


name                          0
planet_status                 0
mass                          9
mass_sini                  1083
radius                        0
orbital_period               14
semi_major_axis              28
eccentricity                242
inclination                 154
temp_calculated              14
temp_measured              1157
detection_type                0
mass_measurement_type       128
radius_measurement_type     128
molecules                  1108
star_name                     0
star_distance                63
star_mass                    46
star_radius                  48
star_teff                    51
mass_final                    0
temp_final                    0
dtype: int64
temp_measured              98.050847
molecules                  93.898305
mass_sini                  91.779661
eccentricity               20.508475
inclination                13.050847
radius_measurement_type    10.847458
mass_measurement_type      10.847458
star_distance           

In [168]:
exoplanetas = exoplanetas.drop(columns=["mass_sini", "temp_measured"])

categoricas = ["molecules", "mass_measurement_type", "radius_measurement_type"]
for col in categoricas:
    exoplanetas[col] = exoplanetas[col].fillna("Unknown")

numericas = ["eccentricity", "inclination", "semi_major_axis", "orbital_period",
             "star_distance", "star_mass", "star_radius", "star_teff"]
for col in numericas:
    exoplanetas[col] = exoplanetas[col].fillna(exoplanetas[col].median())

print("✅ Missing values restantes por columna:\n", exoplanetas.isna().sum())

✅ Missing values restantes por columna:
 name                        0
planet_status               0
mass                        9
radius                      0
orbital_period              0
semi_major_axis             0
eccentricity                0
inclination                 0
temp_calculated            14
detection_type              0
mass_measurement_type       0
radius_measurement_type     0
molecules                   0
star_name                   0
star_distance               0
star_mass                   0
star_radius                 0
star_teff                   0
mass_final                  0
temp_final                  0
dtype: int64


In [169]:
# Columnas numéricas que queremos imputar (mediana)
numericas = ["eccentricity", "inclination", "semi_major_axis", "orbital_period",
             "star_distance", "star_mass", "star_radius", "star_teff", "temp_calculated"]

for col in numericas:
    exoplanetas[col] = exoplanetas[col].fillna(exoplanetas[col].median())

# Columnas categóricas
categoricas = ["molecules", "mass_measurement_type", "radius_measurement_type"]
for col in categoricas:
    exoplanetas[col] = exoplanetas[col].fillna("Unknown")

# Verificar missing values finales
print("✅ Missing values restantes por columna:\n", exoplanetas.isna().sum())


✅ Missing values restantes por columna:
 name                       0
planet_status              0
mass                       9
radius                     0
orbital_period             0
semi_major_axis            0
eccentricity               0
inclination                0
temp_calculated            0
detection_type             0
mass_measurement_type      0
radius_measurement_type    0
molecules                  0
star_name                  0
star_distance              0
star_mass                  0
star_radius                0
star_teff                  0
mass_final                 0
temp_final                 0
dtype: int64


Renombre y normalización de las columnas del CSV de Exoplanetas

In [170]:
exoplanetas = exoplanetas.rename(columns={
    "name": "pl_name",
    "detection_type": "discoverymethod",
    "orbital_period": "pl_orbper",
    "mass": "pl_bmasse",
    "radius": "pl_rade",
    "star_name": "hostname",
    "star_distance": "sy_dist",
    "star_mass": "st_mass",
    "star_radius": "st_rad",
    "star_teff": "st_teff",
    "semi_major_axis": "pl_orbsmax",
    "eccentricity": "pl_orbeccen",
    "temp_calculated": "pl_eqt"
})


In [171]:
for exoplanetas in [planetas, exoplanetas]: 
    exoplanetas['pl_name'] = exoplanetas['pl_name'].str.strip().str.upper()

exoplanetas = exoplanetas.groupby("pl_name", as_index=False).first()

Limpieza de tildes en nombres de columnas y del contenido

Convertí las columnas a categóricas, para poder normalizar todo

In [172]:
exoplanetas_final = exoplanetas_final.astype(str)

# 2. Quitar tildes en columnas
exoplanetas_final.columns = [
    unicodedata.normalize("NFKD", c).encode("ascii", "ignore").decode("ascii")
    for c in exoplanetas_final.columns
]

# 3. Limpiar contenido sin errores de float
exoplanetas_final = exoplanetas_final.map(lambda x:
    unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("ascii")
    .strip()
    .replace('"','')
    .replace("'",'')
)

Convertí nuevamente las columnas en numéricas

In [173]:
columnas_numericas = [
    "masa_exoplaneta_m_tierra",
    "radio_exoplaneta_r_tierra",
    "periodo_orbital_dias",
    "semi_eje_mayor_ua",
    "excentricidad_orbital",
    "temperatura_media_k",
    "insolacion"
]

# Convertirlas a float
exoplanetas_final[columnas_numericas] = exoplanetas_final[columnas_numericas].astype(float)

print(exoplanetas_final.dtypes)  # para comprobar que ya son numeros


nombre_exoplaneta             object
estado_exoplaneta             object
masa_exoplaneta_m_tierra     float64
radio_exoplaneta_r_tierra    float64
periodo_orbital_dias         float64
semi_eje_mayor_ua            float64
excentricidad_orbital        float64
temperatura_media_k          float64
insolacion                   float64
nombre_estrella               object
dtype: object


#### Unión del CSV 'Planetas' y 'Exoplanetas'

In [174]:
exoplanetas_final = planetas.merge(exoplanetas, on="pl_name", how="outer")

Rellenar valores nulos

In [175]:
text_cols = exoplanetas_final.select_dtypes(include=["object"]).columns
exoplanetas_final[text_cols] = exoplanetas_final[text_cols].fillna("Unknown")

# 🔹 Columnas numéricas → rellenar por la media si hay duplicados según tipo de planeta
num_cols = exoplanetas_final.select_dtypes(include=[np.number]).columns
exoplanetas_final[num_cols] = exoplanetas_final[num_cols].fillna(exoplanetas_final[num_cols].mean())

#### Elección de las columnas importantes para cada uno de mis DataFrame

In [ ]:
# --- 1️⃣ DataFrame de estrellas (único por estrella)
cols_estrellas = [
    'hostname_x', 'st_mass_x', 'st_rad_x', 'st_teff_x', 'sy_dist_x'
]

estrellas_exoplanetas = exoplanetas_final[cols_estrellas].drop_duplicates(subset='hostname_x').copy()

# Renombrar columnas para más claridad
estrellas_exoplanetas = estrellas_exoplanetas.rename(columns={
    'hostname_x': 'star_name',
    'st_mass_x': 'star_mass',
    'st_rad_x': 'star_radius',
    'st_teff_x': 'star_teff',
    'sy_dist_x': 'star_distance'
})

# --- 2️⃣ DataFrame de exoplanetas con sus columnas importantes
cols_planetas = [
    'pl_name', 'planet_status',
    'pl_bmasse_x', 'pl_rade_x', 'pl_orbper_x',
    'pl_orbsmax_x', 'pl_orbeccen_x', 'pl_eqt_x', 'pl_insol',
    'hostname_x'
]

exoplanetas_final = exoplanetas_final[cols_planetas].copy()

# Renombrar columnas para simplificar nombres
exoplanetas_final = exoplanetas_final.rename(columns={
    'pl_bmasse_x': 'pl_mass', 
    'pl_rade_x': 'pl_radius',
    'pl_orbper_x': 'pl_orbital_period',
    'pl_orbsmax_x': 'pl_semi_major_axis',
    'pl_orbeccen_x': 'pl_eccentricity',
    'pl_eqt_x': 'pl_eq_temp',
    'hostname_x': 'star_name'
})

# --- Resultado
print("✅ DataFrame estrellas")
print(estrellas_exoplanetas.head())

print("\n✅ DataFrame exoplanetas")
print(exoplanetas_final.head())


✅ DataFrame estrellas
   star_name  star_mass  star_radius   star_teff  star_distance
0     11 Com       2.09    13.760000  4874.00000        93.1846
3     11 UMi       1.80    24.080000  4340.00000       125.3210
6     14 And       1.20     1.152725  5462.42793        75.4392
9     14 Her       0.91     1.152725  5462.42793        17.9323
17  16 Cyg B       1.08     1.130000  5750.00000        21.1397

✅ DataFrame exoplanetas
    pl_name planet_status      pl_mass  pl_radius  pl_orbital_period  \
0  11 COM B       Unknown  4914.898486   5.443606         323.210000   
1  11 COM B       Unknown  6165.600000   5.443606         326.030000   
2  11 COM B       Unknown  5434.700000   5.443606       12207.906655   
3  11 UMI B       Unknown  3337.070000   5.443606         516.220000   
4  11 UMI B       Unknown  4684.814200   5.443606         516.219970   

   pl_semi_major_axis  pl_eccentricity  pl_eq_temp    pl_insol star_name  
0               1.178          0.23800   879.81474  362.91254

Agrupar todos los nombres de los planetas, por sus características propias, sin duplicados

In [177]:
exoplanetas_final = exoplanetas_final.groupby('pl_name').agg({
    'planet_status': 'first',
    'pl_mass': 'mean',
    'pl_radius': 'mean',
    'pl_orbital_period': 'mean',
    'pl_semi_major_axis': 'mean',
    'pl_eccentricity': 'mean',
    'pl_eq_temp': 'mean',
    'pl_insol': 'mean',
    'star_name': 'first'
}).reset_index()


Filtrado solo por los planetas 'Confirmados'

In [178]:
# Filtrar solo planetas confirmados
exoplanetas_confirmados = exoplanetas_final[exoplanetas_final['planet_status'] == 'Confirmed'].copy()

# Ver cuántos quedan
print(f"Filas restantes: {len(exoplanetas_confirmados)}")
print(exoplanetas_confirmados.head())


Filas restantes: 1180
                   pl_name planet_status      pl_mass  pl_radius  \
12  2MASS J0103-5515 (AB)B     Confirmed   727.468962   5.443606   
30               51 ERI AB     Confirmed   727.468962   5.443606   
54                AF LEP B     Confirmed  1298.885242   5.443606   
58                AU MIC B     Confirmed   277.209568   4.828064   
59                AU MIC C     Confirmed   218.019704   4.332349   

    pl_orbital_period  pl_semi_major_axis  pl_eccentricity   pl_eq_temp  \
12       12207.906655            4.698996         0.046370   879.814740   
30       12207.906655            4.698996         0.046370   879.814740   
54        8378.750000            8.671500         0.261250  1009.861055   
58           8.462271            2.592571         0.057932   816.921629   
59          18.859077            2.076855         0.044784   758.867671   

      pl_insol star_name  
12  362.912545   Unknown  
30  362.912545   Unknown  
54  362.912545    AF Lep  
58  305.82

Añadido las características de la Tierra

In [179]:
# Asumiendo exoplanetas_confirmados como DataFrame limpio
exoplanetas_confirmados['mass_rel_earth'] = exoplanetas_confirmados['pl_mass'] / 1
exoplanetas_confirmados['radius_rel_earth'] = exoplanetas_confirmados['pl_radius'] / 1
exoplanetas_confirmados['orbital_period_rel_earth'] = exoplanetas_confirmados['pl_orbital_period'] / 365
exoplanetas_confirmados['semi_major_axis_rel_earth'] = exoplanetas_confirmados['pl_semi_major_axis'] / 1
exoplanetas_confirmados['flux_rel_earth'] = exoplanetas_confirmados['pl_insol'] / 1


Añadido las características del Sol

In [180]:
estrellas_exoplanetas['mass_rel_sun'] = estrellas_exoplanetas['star_mass'] / 1  # M☉
estrellas_exoplanetas['radius_rel_sun'] = estrellas_exoplanetas['star_radius'] / 1  # R☉
estrellas_exoplanetas['teff_rel_sun'] = estrellas_exoplanetas['star_teff'] / 5778  # K del Sol


Guardar en un CSV cada uno

In [181]:
# Guardar estrellas
estrellas_exoplanetas.to_csv("estrellas_exoplanetas.csv", index=False)

# Guardar exoplanetas
exoplanetas_confirmados.to_csv("exoplanetas_final.csv", index=False)

print("✅ CSVs guardados: 'estrellas_exoplanetas.csv' y 'exoplanetas_final.csv'")


✅ CSVs guardados: 'estrellas_exoplanetas.csv' y 'exoplanetas_final.csv'


## CSV Planetas, estrellas anfitrionas y candidatos

Filtrar Planetas Candidatos por confirmados

In [182]:
planetas_candidatos_confirmados = planetas_candidatos[planetas_candidatos['disposition'] == 'CONFIRMED']

Concatenar Planetas_candidatos_confirmados y Planetas

In [183]:
planetas_todos = pd.concat([planetas, planetas_candidatos_confirmados], ignore_index=True)

Elegir columnas útiles

In [184]:
columnas_planetas = [
    'pl_name', 'hostname', 'pl_orbper', 'pl_orbsmax', 'pl_rade', 'pl_bmasse',
    'pl_orbeccen', 'pl_eqt', 'st_teff', 'st_mass', 'st_rad', 'st_spectype', 'sy_dist'
]
planetas_todos = planetas_todos[columnas_planetas]

#### Imputar missing values 
- Numéricas (pl_orbper, pl_orbsmax, pl_rade, pl_bmasse, pl_orbeccen, pl_eqt, st_teff, st_mass, st_rad, sy_dist) → mediana

- Categóricas (st_spectype) → 'Unknown'

Resetear Índice

In [185]:
planetas_todos = planetas_todos.reset_index(drop=True)

Unión de este Dataset con Exoplanetas, añadiendo la columna de tipo espectral

In [ ]:
columnas_nuevas = ['pl_name', 'st_spectype']  
exoplanetas_extra = planetas_todos[columnas_nuevas].drop_duplicates(subset='pl_name')


In [187]:
exoplanetas_final = exoplanetas_final.merge(
    exoplanetas_extra,
    on='pl_name',
    how='left'
)


Rellenar los valores nulos con 'Unknown'

In [188]:
exoplanetas_final['st_spectype'] = exoplanetas_final['st_spectype'].fillna('Unknown')


Solo coger los exoplanetas confirmados

In [189]:
# Filtrar solo planetas confirmados
exoplanetas_final = exoplanetas_final[
    exoplanetas_final['planet_status'] == 'Confirmed'
].copy()

# Verificar cuántas filas quedan
print(f"Planetas confirmados: {len(exoplanetas_final)}")
print(exoplanetas_final[['pl_name', 'planet_status']].head())


Planetas confirmados: 1180
                   pl_name planet_status
12  2MASS J0103-5515 (AB)B     Confirmed
30               51 ERI AB     Confirmed
54                AF LEP B     Confirmed
58                AU MIC B     Confirmed
59                AU MIC C     Confirmed


##### Cambio los nombres de las columnas

In [ ]:
renombrar_columnas = {
    'pl_name': 'nombre_exoplaneta',
    'planet_status': 'estado_exoplaneta',
    'pl_mass': 'masa_exoplaneta_m_tierra',          
    'pl_radius': 'radio_exoplaneta_r_tierra',      
    'pl_orbital_period': 'periodo_orbital_dias',
    'pl_semi_major_axis': 'semi_eje_mayor_ua',
    'pl_eccentricity': 'excentricidad_orbital',
    'pl_eq_temp': 'temperatura_media_k',
    'pl_insol': 'insolacion',                      
    'star_name': 'nombre_estrella',
    'st_spectype': 'tipo_espectral'
}


exoplanetas_final.rename(columns=renombrar_columnas, inplace=True)

CSV completo con todos los CSV de los Exoplanetas

In [191]:
exoplanetas_final.to_csv('exoplanetas_final.csv', index=False)

In [192]:
estrellas_nuevas = estrellas_anfitrionas[['sy_name', 'st_teff', 'st_mass', 'st_rad', 'st_spectype', 'sy_dist']].copy()


In [193]:
estrellas_nuevas = estrellas_nuevas.drop_duplicates(subset='sy_name')


In [ ]:
estrellas_exoplanetas = estrellas_exoplanetas.merge(
    estrellas_nuevas,
    left_on='star_name',   
    right_on='sy_name',
    how='left'
)


In [195]:
columnas_utiles = [
    'star_name', 'star_mass', 'star_radius', 'star_teff', 'star_distance',
    'mass_rel_sun', 'radius_rel_sun', 'teff_rel_sun'
]

estrellas_exoplanetas = estrellas_exoplanetas[columnas_utiles].copy()


##### Cambio de nombre de las columnas

In [ ]:
renombrar_columnas_estrellas = {
    'star_name': 'nombre_estrella',
    'star_mass': 'masa_estrella_m_solar',        
    'star_radius': 'radio_estrella_r_solar',    
    'star_teff': 'temperatura_estrella_k',      
    'star_distance': 'distancia_estrella_pc',   
    'mass_rel_sun': 'masa_rel_solar',           
    'radius_rel_sun': 'radio_rel_solar',        
    'teff_rel_sun': 'temperatura_rel_solar'     
}

# Renombrar columnas
estrellas_exoplanetas.rename(columns=renombrar_columnas_estrellas, inplace=True)

Redondear todos los valores numéricos a 2 decimales

In [197]:
cols_numericas = [
    'masa_estrella_m_solar', 'radio_estrella_r_solar', 'temperatura_estrella_k',
    'distancia_estrella_pc', 'masa_rel_solar', 'radio_rel_solar', 'temperatura_rel_solar'
]

# Redondear a 2 decimales
estrellas_exoplanetas[cols_numericas] = estrellas_exoplanetas[cols_numericas].round(2)

Sobreescribir el CSV que ya tenía

In [198]:
estrellas_exoplanetas.to_csv("estrellas_exoplanetas.csv", index=False)


### Creación de los datos de la Tierra

In [199]:
# Crear un diccionario con los datos de la Tierra
tierra = {
      # Datos para relacionar con planetas
    'nombre_planeta': 'Tierra',
    'nombre_estrella': 'Sol',
    'similitud_tierra': 1.0,
    'masa_planeta_m_tierra': 1.0,         # Masa terrestre en M⊕ (Earth Mass)
    'radio_planeta_r_tierra': 1.0,        # Radio terrestre en R⊕ (Earth Radius)
    'temperatura_media_k': 288,           # Temperatura media en K
    'periodo_orbital_dias': 365.25,       # Periodo orbital en días
    'semi_eje_mayor_ua': 1.0,             # Semi-eje mayor en UA
    'excentricidad_orbital': 0.0167,      # Excentricidad orbital

    # Datos para relacionar con estrellas
    'temperatura_estrella_k': 5778,       # Temperatura efectiva de la estrella (K)
    'masa_estrella_m_solar': 1.0,         # Masa solar en M☉
    'radio_estrella_r_solar': 1.0,        # Radio solar en R☉
    'tipo_espectral': 'G2V',              # Tipo espectral

    # Datos para misiones (opcional)
    'origen_mision': 'Tierra'             # Nombre del planeta de origen de las misiones
}

# Convertir a DataFrame
tierra_df = pd.DataFrame([tierra])

# Guardar como CSV listo para SQL
tierra_df.to_csv("tierra.csv", index=False)

print("CSV de la Tierra creado correctamente con todas las columnas de referencia.")

CSV de la Tierra creado correctamente con todas las columnas de referencia.


In [200]:
tierra_df.to_csv("tierra.csv", index=False)

### Creación Sistema Solar

In [201]:
# Datos del Sistema Solar con todas las lunas conocidas hasta la actualidad
planetas_sistema_solar_completo = {
    "Nombre": ["Mercurio", "Venus", "Tierra", "Marte", "Júpiter", "Saturno", "Urano", "Neptuno"],
    "Masa_kg": [3.30e23, 4.87e24, 5.97e24, 6.42e23, 1.90e27, 5.68e26, 8.68e25, 1.02e26],
    "Radio_km": [2439.7, 6051.8, 6371, 3389.5, 69911, 58232, 25362, 24622],
    "Densidad_kg_m3": [5427, 5243, 5514, 3933, 1326, 687, 1270, 1638],
    "Gravedad_m_s2": [3.7, 8.87, 9.807, 3.721, 24.79, 10.44, 8.69, 11.15],
    "Distancia_solar_AU": [0.39, 0.72, 1.0, 1.52, 5.2, 9.58, 19.2, 30.05],
    "Periodo_orbital_dias": [88, 225, 365.25, 687, 4331, 10747, 30589, 59800],
    "Inclinacion_orbital_grados": [7.0, 3.39, 0.0, 1.85, 1.3, 2.49, 0.77, 1.77],
    "Numero_lunas": [0, 0, 1, 2, 79, 82, 27, 14],
    "Nombres_lunas": [
        "-", 
        "-", 
        "Luna", 
        "Fobos, Deimos", 
        "Io, Europa, Ganimedes, Calisto, Amaltea, Himalia, Elara, Pasiphae, Sinope, Lysithea, Carme, Ananke, Leda, ...", 
        "Titán, Rea, Jápeto, Dione, Tetis, Mimas, Encélado, Hiperión, Febe, Pan, Atlas, Prometeo, Pandora, ...", 
        "Titania, Oberón, Umbriel, Ariel, Miranda, Puck, Portia, Rosalind, Belinda, Cressida, Desdemona, Juliet, ...", 
        "Tritón, Nereida, Proteo, Larisa, Galatea, Despina, Thalassa, Naiad, Halimede, Psámate, Sao, Laomedeia, Neso, ..."],
    "Tipo_planeta": ["Rocoso", "Rocoso", "Rocoso", "Rocoso", "Gigante gaseoso", "Gigante gaseoso", "Gigante helado", "Gigante helado"],
    "Temperatura_media_K": [440, 737, 288, 210, 165, 134, 76, 72]
}

# Crear DataFrame
df_planetas_solar_completo = pd.DataFrame(planetas_sistema_solar_completo)


In [202]:
# Diccionario para renombrar columnas
renombrar_columnas = {
    "Nombre": "nombre_planeta",
    "Masa_kg": "masa_kg",
    "Radio_km": "radio_km",
    "Densidad_kg_m3": "densidad_kg_m3",
    "Gravedad_m_s2": "gravedad_m_s2",
    "Distancia_solar_AU": "distancia_sol_au",
    "Periodo_orbital_dias": "periodo_orbital_dias",
    "Inclinacion_orbital_grados": "inclinacion_orbital_grados",
    "Numero_lunas": "numero_lunas",
    "Nombres_lunas": "nombres_lunas",
    "Tipo_planeta": "tipo_planeta",
    "Temperatura_media_K": "temperatura_media_k"
}

# Renombrar columnas
df_planetas_solar_completo.rename(columns=renombrar_columnas, inplace=True)


In [203]:
df_planetas_solar_completo.columns

Index(['nombre_planeta', 'masa_kg', 'radio_km', 'densidad_kg_m3',
       'gravedad_m_s2', 'distancia_sol_au', 'periodo_orbital_dias',
       'inclinacion_orbital_grados', 'numero_lunas', 'nombres_lunas',
       'tipo_planeta', 'temperatura_media_k'],
      dtype='object')

In [204]:
# 1️⃣ Convertir TODO a string temporalmente
df_planetas_solar_completo = df_planetas_solar_completo.astype(str)

# 2️⃣ Limpiar nombres de columnas
df_planetas_solar_completo.columns = [
    unicodedata.normalize("NFKD", c).encode("ascii","ignore").decode("ascii")
    for c in df_planetas_solar_completo.columns
]

# 3️⃣ Limpiar todo el contenido textual
df_planetas_solar_completo = df_planetas_solar_completo.applymap(lambda x: 
    unicodedata.normalize("NFKD", x).encode("ascii","ignore").decode("ascii").strip()
    .replace('"','')
    .replace("'",'')
)

# 4️⃣ Convertir columnas numéricas a float otra vez
cols_float = [
    "masa_kg", "radio_km", "densidad_kg_m3", "gravedad_m_s2", "distancia_sol_au",
    "periodo_orbital_dias", "inclinacion_orbital_grados", "numero_lunas",
    "temperatura_media_k"
]

df_planetas_solar_completo[cols_float] = df_planetas_solar_completo[cols_float].astype(float)

# 5️⃣ Guardar CSV limpio listo para SQL
df_planetas_solar_completo.to_csv("planetas_sistema_solar_limpio.csv", index=False, encoding="utf-8")
print("✔ CSV limpio y listo para SQL")


✔ CSV limpio y listo para SQL


C:\Users\CeX-Laguna\AppData\Local\Temp\ipykernel_15748\1741815553.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_planetas_solar_completo = df_planetas_solar_completo.applymap(lambda x:


#### Creación CSV de los satélites del Sistema Solar

In [206]:
df_planetas_solar_completo

,nombre_planeta,masa_kg,radio_km,densidad_kg_m3,gravedad_m_s2,distancia_sol_au,periodo_orbital_dias,inclinacion_orbital_grados,numero_lunas,nombres_lunas,tipo_planeta,temperatura_media_k
0,Mercurio,3.300000e+23,2439.7,5427.0,3.700,0.39,88.00,7.00,0.0,-,Rocoso,440.0
1,Venus,4.870000e+24,6051.8,5243.0,8.870,0.72,225.00,3.39,0.0,-,Rocoso,737.0
2,Tierra,5.970000e+24,6371.0,5514.0,9.807,1.00,365.25,0.00,1.0,Luna,Rocoso,288.0
3,Marte,6.420000e+23,3389.5,3933.0,3.721,1.52,687.00,1.85,2.0,"Fobos, Deimos",Rocoso,210.0
4,Jupiter,1.900000e+27,69911.0,1326.0,24.790,5.20,4331.00,1.30,79.0,"Io, Europa, Ganimedes, Calisto, Amaltea, Himalia, Elara, Pasiphae, Sinope, Lysithea, Carme, Ananke, Leda, ...",Gigante gaseoso,165.0
5,Saturno,5.680000e+26,58232.0,687.0,10.440,9.58,10747.00,2.49,82.0,"Titan, Rea, Japeto, Dione, Tetis, Mimas, Encelado, Hiperion, Febe, Pan, Atlas, Prometeo, Pandora, ...",Gigante gaseoso,134.0
6,Urano,8.680000e+25,25362.0,1270.0,8.690,19.20,30589.00,0.77,27.0,"Titania, Oberon, Umbriel, Ariel, Miranda, Puck, Portia, Rosalind, Belinda, Cressida, Desdemona, Juliet, ...",Gigante helado,76.0
7,Neptuno,1.020000e+26,24622.0,1638.0,11.150,30.05,59800.00,1.77,14.0,"Triton, Nereida, Proteo, Larisa, Galatea, Despina, Thalassa, Naiad, Halimede, Psamate, Sao, Laomedeia, Neso, ...",Gigante helado,72.0


In [211]:
datos_satelites = []

for idx, row in df_planetas_solar_completo.iterrows():
    if row['numero_lunas'] > 0 and row['nombres_lunas'] != "-":
        lunas = [l.strip() for l in row['nombres_lunas'].split(',')]
        for luna in lunas:
            datos_satelites.append({
                'Nombre_Luna': luna,
                'Planeta_Anfitrion': row['nombre_planeta'],
                'Tipo_Planeta': row['tipo_planeta']
            })

df_lunas = pd.DataFrame(datos_satelites)
df_lunas.to_csv("satélites_sistema_solar.csv", index=False)
print("CSV de todas las lunas separado creado correctamente.")

CSV de todas las lunas separado creado correctamente.


In [212]:
# Datos de las lunas más importantes del Sistema Solar
lunas = [
    {"Nombre_Luna": "Luna", "Planeta_Anfitrion": "Tierra", "Tipo_Planeta": "Rocoso",
     "Masa": 7.34767309e22, "Radio": 1737.1, "Densidad": 3340, "Gravedad": 1.62,
     "Periodo_orbital": 27.32, "Distancia_planeta": 384400, "Excentricidad": 0.0549,
     "Descubierta": "Prehistorico", "Descubridor": "Desconocido"},
    {"Nombre_Luna": "Fobos", "Planeta_Anfitrion": "Marte", "Tipo_Planeta": "Rocoso",
     "Masa": 1.0659e16, "Radio": 11.2667, "Densidad": 1876, "Gravedad": 0.0057,
     "Periodo_orbital": 0.31891, "Distancia_planeta": 9376, "Excentricidad": 0.0151,
     "Descubierta": 1877, "Descubridor": "Asaph Hall"},
    {"Nombre_Luna": "Deimos", "Planeta_Anfitrion": "Marte", "Tipo_Planeta": "Rocoso",
     "Masa": 1.4762e15, "Radio": 6.2, "Densidad": 1750, "Gravedad": 0.003, 
     "Periodo_orbital": 1.263, "Distancia_planeta": 23463.2, "Excentricidad": 0.00033,
     "Descubierta": 1877, "Descubridor": "Asaph Hall"},
    {"Nombre_Luna": "Io", "Planeta_Anfitrion": "Jupiter", "Tipo_Planeta": "Gigante gaseoso",
     "Masa": 8.9319e22, "Radio": 1821.6, "Densidad": 3530, "Gravedad": 1.796,
     "Periodo_orbital": 1.769, "Distancia_planeta": 421700, "Excentricidad": 0.0041,
     "Descubierta": 1610, "Descubridor": "Galileo Galilei"},
    {"Nombre_Luna": "Europa", "Planeta_Anfitrion": "Jupiter", "Tipo_Planeta": "Gigante gaseoso",
     "Masa": 4.7998e22, "Radio": 1560.8, "Densidad": 3010, "Gravedad": 1.314,
     "Periodo_orbital": 3.551, "Distancia_planeta": 671034, "Excentricidad": 0.009,
     "Descubierta": 1610, "Descubridor": "Galileo Galilei"},
    {"Nombre_Luna": "Ganimedes", "Planeta_Anfitrion": "Jupiter", "Tipo_Planeta": "Gigante gaseoso",
     "Masa": 1.4819e23, "Radio": 2634.1, "Densidad": 1940, "Gravedad": 1.428,
     "Periodo_orbital": 7.155, "Distancia_planeta": 1070400, "Excentricidad": 0.0013,
     "Descubierta": 1610, "Descubridor": "Galileo Galilei"},
    {"Nombre_Luna": "Calisto", "Planeta_Anfitrion": "Jupiter", "Tipo_Planeta": "Gigante gaseoso",
     "Masa": 1.0759e23, "Radio": 2410.3, "Densidad": 1830, "Gravedad": 1.235,
     "Periodo_orbital": 16.689, "Distancia_planeta": 1882700, "Excentricidad": 0.0074,
     "Descubierta": 1610, "Descubridor": "Galileo Galilei"},
    {"Nombre_Luna": "Titan", "Planeta_Anfitrion": "Saturno", "Tipo_Planeta": "Gigante gaseoso",
     "Masa": 1.3452e23, "Radio": 2575, "Densidad": 1881, "Gravedad": 1.352, 
     "Periodo_orbital": 15.945, "Distancia_planeta": 1221870, "Excentricidad": 0.0288,
     "Descubierta": 1655, "Descubridor": "Christiaan Huygens"},
    {"Nombre_Luna": "Titania", "Planeta_Anfitrion": "Urano", "Tipo_Planeta": "Gigante helado",
     "Masa": 3.527e21, "Radio": 788.9, "Densidad": 1700, "Gravedad": 0.379, 
     "Periodo_orbital": 8.706, "Distancia_planeta": 436300, "Excentricidad": 0.0011,
     "Descubierta": 1787, "Descubridor": "William Herschel"},
    {"Nombre_Luna": "Triton", "Planeta_Anfitrion": "Neptuno", "Tipo_Planeta": "Gigante helado",
     "Masa": 2.14e22, "Radio": 1353.4, "Densidad": 2050, "Gravedad": 0.779, 
     "Periodo_orbital": -5.877, "Distancia_planeta": 354800, "Excentricidad": 0.000016,
     "Descubierta": 1846, "Descubridor": "William Lassell"}
]


# Convertir a DataFrame
df_lunas_sistema_solar = pd.DataFrame(lunas)

# Guardar CSV listo para SQL
df_lunas_sistema_solar.to_csv("lunas_sistema_solar_completo.csv", index=False)

print("CSV de lunas del Sistema Solar creado correctamente con datos reales.")


CSV de lunas del Sistema Solar creado correctamente con datos reales.


#### Creación de las características del Sol

In [213]:
sol = {
    "Nombre": ["Sol"],
    "Masa_kg": [1.989e30],
    "Radio_km": [696340],
    "Densidad_media_kg_m3": [1408],
    "Temperatura_superficial_K": [5778],
    "Temperatura_central_K": [1.57e7],
    "Composicion_H_%": [73],
    "Composicion_He_%": [25],
    "Composicion_Otros_%": [2],
    "Gravedad_superficial_m_s2": [274],
    "Magnitud_absoluta": [4.83],
    "Metallicidad_Fe_H_dex": [0],
    "Tipo_espectral": ["G2V"]
}

# Crear DataFrame
df_sol = pd.DataFrame(sol)

# Guardar en CSV
df_sol.to_csv("sol.csv", index=False)

print("CSV del Sol creado correctamente.")


CSV del Sol creado correctamente.


In [214]:
corr = exoplanetas_final.corr(numeric_only=True)
print(corr)


                           masa_exoplaneta_m_tierra  \
masa_exoplaneta_m_tierra                   1.000000   
radio_exoplaneta_r_tierra                  0.308332   
periodo_orbital_dias                       0.157201   
semi_eje_mayor_ua                          0.291767   
excentricidad_orbital                      0.090637   
temperatura_media_k                        0.102560   
insolacion                                 0.089728   

                           radio_exoplaneta_r_tierra  periodo_orbital_dias  \
masa_exoplaneta_m_tierra                    0.308332              0.157201   
radio_exoplaneta_r_tierra                   1.000000              0.039582   
periodo_orbital_dias                        0.039582              1.000000   
semi_eje_mayor_ua                           0.068110              0.881769   
excentricidad_orbital                       0.105760             -0.007343   
temperatura_media_k                         0.483117             -0.027329   
insolacion   

Con esto, se demuestra que, cuando un exoplaneta recibe masenergia de su estrella, su temperatura suele ser mayor. Con lo que la conclusión es cuanta más energía recibe, más caliente es el planeta

In [215]:
corr = estrellas_exoplanetas.corr(numeric_only=True)
print(corr)

                        masa_estrella_m_solar  radio_estrella_r_solar  \
masa_estrella_m_solar                1.000000                0.330282   
radio_estrella_r_solar               0.330282                1.000000   
temperatura_estrella_k               0.207392               -0.068735   
distancia_estrella_pc               -0.169361               -0.035282   
masa_rel_solar                       1.000000                0.330282   
radio_rel_solar                      0.330282                1.000000   
temperatura_rel_solar                0.206913               -0.068771   

                        temperatura_estrella_k  distancia_estrella_pc  \
masa_estrella_m_solar                 0.207392              -0.169361   
radio_estrella_r_solar               -0.068735              -0.035282   
temperatura_estrella_k                1.000000               0.100218   
distancia_estrella_pc                 0.100218               1.000000   
masa_rel_solar                        0.207392    

- La masa y el radio de las estrellas tienen una relación débil positiva (0.33): estrellas más masivas tienden a ser un poco más grandes.

- La masa y la temperatura muestran también una relación débil positiva (0.21): estrellas más masivas suelen ser ligeramente más calientes.

- La distancia de la estrella no se relaciona con ninguna propiedad física (correlación cercana a 0).

In [216]:
corr = df_planetas_solar_completo.corr(numeric_only=True)
print(corr)

                             masa_kg  radio_km  densidad_kg_m3  gravedad_m_s2  \
masa_kg                     1.000000  0.867638       -0.524033       0.919074   
radio_km                    0.867638  1.000000       -0.819154       0.814549   
densidad_kg_m3             -0.524033 -0.819154        1.000000      -0.510499   
gravedad_m_s2               0.919074  0.814549       -0.510499       1.000000   
distancia_sol_au           -0.049959  0.253745       -0.672289       0.135693   
periodo_orbital_dias       -0.126373  0.142559       -0.563781       0.077741   
inclinacion_orbital_grados -0.205068 -0.286341        0.385920      -0.413241   
numero_lunas                0.812108  0.979270       -0.792245       0.706959   
temperatura_media_k        -0.285735 -0.506413        0.766672      -0.270123   

                            distancia_sol_au  periodo_orbital_dias  \
masa_kg                            -0.049959             -0.126373   
radio_km                            0.253745     

- Masa y radio: alta correlación positiva (0.87), es decir, los planetas más masivos tienden a ser más grandes.

- Masa y gravedad: muy alta correlación positiva (0.92), lo esperado: más masa → mayor gravedad superficial.

- Radio y densidad: fuerte correlación negativa (-0.82), los planetas más grandes tienden a ser menos densos (como Júpiter o Saturno).

- Número de lunas: correlaciona positivamente con masa (0.81) y radio (0.98), los planetas grandes tienen más lunas.

- Distancia al Sol y período orbital: casi perfecta correlación positiva (0.99), siguiendo la tercera ley de Kepler: cuanto más lejos del Sol, mayor período orbital.

- Densidad y temperatura media: correlación positiva (0.77), los planetas más densos suelen ser más cálidos.

- Inclinación orbital: correlaciones moderadas con densidad y gravedad (-0.31 a -0.41), indica cierta relación con la forma y orientación de la órbita.

- Distancia al Sol y temperatura media: correlación negativa moderada (-0.63), los planetas más alejados del Sol son más fríos.

In [217]:
corr = df_lunas_sistema_solar.corr(numeric_only=True)
print(corr)

                       Masa     Radio  Densidad  Gravedad  Periodo_orbital  \
Masa               1.000000  0.938962  0.162388  0.798233         0.488115   
Radio              0.938962  1.000000  0.212943  0.863648         0.477660   
Densidad           0.162388  0.212943  1.000000  0.674229         0.193664   
Gravedad           0.798233  0.863648  0.674229  1.000000         0.465338   
Periodo_orbital    0.488115  0.477660  0.193664  0.465338         1.000000   
Distancia_planeta  0.765970  0.817827 -0.185870  0.516745         0.467508   
Excentricidad      0.232724  0.207846  0.416760  0.364201         0.783816   

                   Distancia_planeta  Excentricidad  
Masa                        0.765970       0.232724  
Radio                       0.817827       0.207846  
Densidad                   -0.185870       0.416760  
Gravedad                    0.516745       0.364201  
Periodo_orbital             0.467508       0.783816  
Distancia_planeta           1.000000       0.010697

- Masa y radio: muy alta correlación positiva (0.94). Los planetas más masivos tienden a ser más grandes.

- Masa y gravedad: alta correlación positiva (0.80). Mayor masa → mayor gravedad superficial. 

- Radio y gravedad: alta correlación positiva (0.86). Planetas más grandes suelen tener mayor gravedad.

- Periodo orbital y distancia al planeta: correlación positiva moderada (0.47-0.47), reflejando la tercera ley de Kepler.

- Excentricidad: casi no correlaciona con masa o radio (0.23 y 0.21), pero sí bastante con el periodo orbital (0.78). Planetas con órbitas más excéntricas tienden a tener períodos más largos.

- Densidad: correlaciones bajas con masa y radio, pero positiva con gravedad (0.67), lo que tiene sentido: densidad mayor → gravedad mayor para un mismo radio.

- Distancia al Sol y masa/radio: correlación positiva moderada (0.77-0.82). Los planetas más alejados tienden a ser más grandes y masivos (los gigantes gaseosos).